In [1]:
import transformers
import torch
from transformers import AdamW
from transformers import BertTokenizer,\
BertForSequenceClassification


In [2]:
pretrained_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name)
model = BertForSequenceClassification.from_pretrained(pretrained_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
input_texts = ['This is a positive review.',
			'This is a negative review.']
labels = [1, 0]

input_ids = []
attention_masks = []

for text in input_texts:
	encoded_dict = tokenizer.encode_plus(
		text,
		add_special_tokens=True,
		max_length=128,
		padding='max_length',
		truncation=True,
		return_tensors='pt'
	)

	input_ids.append(encoded_dict['input_ids'])
	attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)


In [4]:
batch_size = 2
epochs = 3
optimizer = AdamW(model.parameters(), lr=2e-5)
model.train()

for epoch in range(epochs):
	for i in range(0, input_ids.size(0), batch_size):
		batch_input_ids = input_ids[i:i+batch_size]
		batch_attention_masks = attention_masks[i:i+batch_size]
		batch_labels = labels[i:i+batch_size]

		optimizer.zero_grad()

		outputs = model(
			input_ids=batch_input_ids,
			attention_mask=batch_attention_masks,
			labels=batch_labels
		)

		loss = outputs.loss
		loss.backward()
		optimizer.step()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [7]:
model.eval()

test_texts = ['This is good review.',
			'This is a bad review.']
test_input_ids = []
test_attention_masks = []

for text in test_texts:
	encoded_dict = tokenizer.encode_plus(
		text,
		add_special_tokens=True,
		max_length=128,
		padding='max_length',
		truncation=True,
		return_tensors='pt'
	)

	test_input_ids.append(encoded_dict['input_ids'])
	test_attention_masks.append(encoded_dict['attention_mask'])

test_input_ids = torch.cat(test_input_ids, dim=0)
test_attention_masks = torch.cat(test_attention_masks, dim=0)

with torch.no_grad():
	outputs = model(
		input_ids=test_input_ids,
		attention_mask=test_attention_masks
	)

predicted_labels = torch.argmax(outputs.logits, dim=1)


In [8]:
for text, label in zip(test_texts, predicted_labels):
	print(f'Text: {text}\nPredicted Label: {label.item()}\n')


Text: This is good review.
Predicted Label: 1

Text: This is a bad review.
Predicted Label: 0

